In [1]:
%load_ext autoreload
%autoreload 2

*Authors: Ema Donev and dr. Ivezić*

In this notebook we tackle the portion of our stars which are RR Lyraes. We use a fine-grid Lomb Scargle periodogram to calculate the periods of `LINEAR` and `ZTF` light curves, after querying for `ZTF` data, to ensure that the `LINEAR` and its `ZTF` counterpart are fully and correctly matched. We use the periodogram given by the Lomb-Scargle method and use it to indicate the star as a possible Blazhko star. Then we proceed to fitting the light curves and calculating rms and chi-2 metrics. We save all of this data and use it for later analysis.

In [2]:
#| code-fold: false
# IMPORTING LIBRARIES
# --------------------

# AstroML & Astropy
from astroML.datasets import fetch_LINEAR_sample
from astropy.timeseries import LombScargle
from astroML.datasets import fetch_LINEAR_sample
from astroML.datasets import fetch_LINEAR_geneva
from astropy.timeseries import TimeSeries
from astropy.table import Table
from astroML.time_series import MultiTermFit

# ZTF
from ztfquery import lightcurve

# Basic libraries
import random
import pickle
import os
import sys
from tqdm import tqdm

# Plotting
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
import matplotlib.colors as mcolors
from matplotlib.font_manager import FontProperties

# DataFrame analysis
import pandas as pd
import dask.dataframe as dd 

# Math libraries
import numpy as np
import scipy as sc
from scipy.stats import norm
from scipy.signal import find_peaks
from array import array

In [3]:
#| code-fold: false
# CONFIG
sns.set_theme() # setting the theme for plotting
np.random.seed(42)

colors = ['#1A090D', '#D8C99B', '#D8973C', '#BD632F', '#273E47']
cmap = mcolors.ListedColormap(colors)

font = FontProperties()
font.set_family('avenir')
font.set_name('Avenir')
font.set_style('normal')
font.set_size('xx-large')

%matplotlib inline  

In [4]:
#| code-fold: false
# Importing custom libraries
# ----------------------------
sys.path.insert(0,'../src/')
from config import*
from descriptive_stats import *
from plots import *

from selection import *
from blazhko_analysis import *

Loading the data!


In [5]:
#| code-fold: false
# LINEAR DATA
data = fetch_LINEAR_sample(data_home='../inputs') # fetching the data from astroML data library

# Methods

The Lomb-Scargle method of computing the period of a variable star using light curves involves fitting multi-level sinusoidal waves to our light curve data over various trial periods, and the period with the most prominent strength or the best fit is the actual period.

This is the formula for the Lomb-Scargle periodogram:

$$P(ω)= \frac{1}{2σ^2}​*(\frac{[∑y_k cosω(t_k−τ)]^2}{∑cos^2 ω(t_k−τ)} + \frac{[∑y_k sin ω(t_k−τ)]^2}{∑ sin^2 ω(t_k−τ)})$$

- $ω$ is the frequency we test to see if it matches our light curve.
- $σ^2$ is the variance of the data. We are dividing by it to normalize the data across all distributions.
- $∑y_k cosω(t_k−τ)$ and $∑y_k sin ω(t_k−τ)$: checking how well the fitted cosine and sine waves align with the data. $t_k$ is the particular time point, and $τ$ is the time delay, for efficiency.
- $∑cos^2 ω(t_k−τ)$ and $∑ sin^2 ω(t_k−τ)$: normalization of the fitting in order to eliminate bias towards powerful or weak wave components. 

We use the Lomb-Scargle periodogram because it is the most famous tool for determining the periodic variable stars' period. It combines multiple methods of analysis, with its core being in Fourier analysis, and it operates with non-uniformly spaced data, such is the nature of light curves.

### Light curve folding (phasing)

**Light curve phasing or folding** is when we condense the entire timeseries of the light curve into one *phase*, or one cycle of variability. Phase is defined with the period of the light curve, as such: $$phase = \frac{time}{period} - int(\frac{time}{period})$$ This function defines the period as a value from 0 to 1 defining how far in is the star in its variability cycle. We can then adjust the `time` array using this function.

### Fitting light curves
**Light curve fitting** requires a sophisticated function which uses Fourier analysis to correctly fit a curve through light curve `time`, `mag` and `magerr` data. We can define how many terms we want our fit to have. *The more terms we use, the more complicated shapes we can graph*. The fitting is an essential part of light curve analysis, as we can then calculate the goodness of fit and monitor how that fit changes over long periods of time, like in the case of `LINEAR` and `ZTF`.

### `Chi-2 test` and `RMS scatter`

#### `RMS scatter`
RMS scatter is a statistical metric that, in this case, measures the goodness of fit versus the actuall data. It is calculated as such: $$RMS = \sqrt{\frac{1}{n}\sum^n_{i=1} x_i^2}$$

#### `Chi-2 test`
This test asseses the "goodness of fit" of light curves, by comparing the fitted values and the actual magnitude values of the light curve. The lower the chi-2 value, the better the correlation between the observed and expected values (based on some model), and the higher the chi-2 value, the poorer the correlation. At which point the value is too high or too low is determined by the statistician, based on the degrees of freedom and the nature of the data. The chi-2 value for every point of a light curve is calculated as such: $$\chi^2 = \sum{\frac{(O_i - E_i)^2}{E_i}}$$

# Selecting RR Lyrae stars

In order to select the RR Lyrae stars, we need information about their classification type and which stars have enough datapoints for correct period calculation. We access data with the `fetch_LINEAR_geneva()` which contains additional information about the light curves in the **Geneva catalog** of the `LINEAR` variable stars.

We have calculated the periods of the RR Lyrae stars beforehand with only 3 Fourier coefficients, enough to differentiate "1 dip" and "2 dip" stars. You can find the entire code for `select_LINEAR` in the `selection.py` file.

In [6]:
#| code-fold: false
L = select_LINEAR('LINEAR_Periods_nterm3')
L.head()

,ID,Porig,Pnew,ra,dec,ug,gi,iK,JK,logP,Ampl,skew,kurt,magMed,nObs,LCtype
0,29848,0.557009,0.557019,119.526443,46.962120,1.17,0.37,1.02,0.27,-0.254138,0.62,-0.31,-0.57,16.37,301,1
1,32086,0.569258,0.569266,119.324013,47.095505,1.36,0.52,1.17,0.31,-0.244691,0.71,-0.49,-1.00,15.02,289,1
2,50402,0.643293,0.643286,119.712975,52.149574,1.18,0.39,1.10,0.20,-0.191591,0.49,-0.29,-0.88,16.46,284,1
3,61011,0.662369,0.662376,118.491257,53.168125,0.81,0.55,1.62,0.20,-0.178900,0.69,-0.03,-1.06,14.08,274,1
4,62892,0.530772,0.530764,119.187241,53.379295,1.12,0.21,1.07,0.21,-0.275092,0.62,-0.55,-0.32,16.54,276,1


In [7]:
L.shape

(5204, 16)

We have now selected all of the "good" `LINEAR` IDs, now we need to select the RR Lyrae stars.

We first select "1 dip stars" using the coefficient between the original (correct) and our calculated period. Since 1 dip stars are easy to calculate periods for, they should match very well (from a range from 0.99 to 1.01). However, Eclipsing Binaries do not match up well, and often times have a ratio where our calculated period is 2 times bigger than the correct one. This is the easiest way to differentiate the two.

In [8]:
#| code-fold: false
# SELECTING 1 dip STARS
# -------------------------

P_ratio = L['Porig']/L['Pnew']

L_1 = L[(P_ratio>0.99)&(P_ratio<1.01)] # where the ratio between the original (correct) and currently calculated periods are 1:1
L_1.head()

,ID,Porig,Pnew,ra,dec,ug,gi,iK,JK,logP,Ampl,skew,kurt,magMed,nObs,LCtype
0,29848,0.557009,0.557019,119.526443,46.962120,1.17,0.37,1.02,0.27,-0.254138,0.62,-0.31,-0.57,16.37,301,1
1,32086,0.569258,0.569266,119.324013,47.095505,1.36,0.52,1.17,0.31,-0.244691,0.71,-0.49,-1.00,15.02,289,1
2,50402,0.643293,0.643286,119.712975,52.149574,1.18,0.39,1.10,0.20,-0.191591,0.49,-0.29,-0.88,16.46,284,1
3,61011,0.662369,0.662376,118.491257,53.168125,0.81,0.55,1.62,0.20,-0.178900,0.69,-0.03,-1.06,14.08,274,1
4,62892,0.530772,0.530764,119.187241,53.379295,1.12,0.21,1.07,0.21,-0.275092,0.62,-0.55,-0.32,16.54,276,1


In [9]:
L_1.shape

(3100, 16)

Now, we select the *RR Lyrae* stars using the color filter values `(g-i)` and the light curve classification `(LCtype)`. 

In [10]:
#| code-fold: false
Lrrlyr = L_1[(L_1['gi']>-0.5)&(L_1['gi']<0.4)&(L_1['LCtype']>0)&(L_1['LCtype']<3)] # RR lyrae type
Lab = Lrrlyr[Lrrlyr['LCtype']==1] # RRab type
Lc = Lrrlyr[Lrrlyr['LCtype']==2] # RRc type
print(len(Lab), len(Lc))
Lrrlyr.head()

2144 566


,ID,Porig,Pnew,ra,dec,ug,gi,iK,JK,logP,Ampl,skew,kurt,magMed,nObs,LCtype
0,29848,0.557009,0.557019,119.526443,46.962120,1.17,0.37,1.02,0.27,-0.254138,0.62,-0.31,-0.57,16.37,301,1
2,50402,0.643293,0.643286,119.712975,52.149574,1.18,0.39,1.10,0.20,-0.191591,0.49,-0.29,-0.88,16.46,284,1
4,62892,0.530772,0.530764,119.187241,53.379295,1.12,0.21,1.07,0.21,-0.275092,0.62,-0.55,-0.32,16.54,276,1
5,91437,0.674728,0.674711,120.294960,40.932457,1.18,0.24,1.09,0.37,-0.170871,0.75,-0.12,-0.93,15.39,177,1
6,95250,0.313869,0.313869,120.124542,40.656620,1.18,-0.14,0.83,0.28,-0.503252,0.55,0.14,-0.65,16.98,222,2


We now have 2710 RR Lyrae stars from the `LINEAR` database. We can differentiate them further into *RR Lyrae AB* and *RR Lyrae C* types, but this can be done later (we just have to conserve the `LCtype` column).

# Analysis of RR Lyrae stars

In this section we showcase the code used for the entire analysis. The definitions of functions can be found in the `lc_analysis.py` file.

In [11]:
#| code-fold: false
# CONFIG BEFORE BIG RUN
Lids = Lrrlyr['ID'].to_numpy() # list of good LINEAR Ids
print(Lids.shape) # checking the shape
lc_type = Lrrlyr['LCtype'].to_numpy()
rectascension = Lrrlyr['ra'].to_numpy() # list of rectascension coordinates
declination = Lrrlyr['dec'].to_numpy() # list of declination coordinates

(2710,)


In [30]:
lc_analysis = {}
fits = []
periodograms = []
end = 'test'
ZTF_data_lc = []
test = len(Lids)
#test = 20

if os.path.isfile('../outputs/LC_ANALYSIS_'+end+'.csv'):
    lightcurve_analysis = pd.read_csv('../outputs/LC_ANALYSIS_'+end+'.csv')
    periodogr = pickle.load(open("../outputs/periodograms_"+end+".pkl", "rb"))
    fits = np.load("../outputs/fits_"+end+".npy", allow_pickle=True)
else:
    for i in tqdm(range(test)):
        # accessing data
        Lid = Lids[i]
        dataL = data.get_light_curve(Lid)

        ra = rectascension[i]
        dec = declination[i]
        ZTFdata = getZTFlightcurve(ra, dec)

        # calculating the periods
        Plinear, fL, pL, tL, mL, meL = LINEARLS(Lids, dataL, i, Lid)
        Pztf, fZ, pZ, tZ, mZ, meZ = ZTFs(ZTFdata, 6, Lid)

        # saving the ZTF data
        ZTF_data_lc.append((Lid, (tZ.to_numpy(), mZ.to_numpy(), meZ.to_numpy())))

        # blazhko periodogram analysis
        fFoldedL, pFoldedL, fMainPeakL, fBlazhkoPeakL, BlazhkoPeriodL, BpowerRatioL, BsignificanceL = getBlazhkoPeak(fL, pL)
        if fZ.size==0 or pZ.size==0:
            fFoldedZ, pFoldedZ, fMainPeakZ, fBlazhkoPeakZ, BlazhkoPeriodZ, BpowerRatioZ, BsignificanceZ = np.array(()), np.array(()), 0, 0, 0, 0, 0
        else:
            fFoldedZ, pFoldedZ, fMainPeakZ, fBlazhkoPeakZ, BlazhkoPeriodZ, BpowerRatioZ, BsignificanceZ = getBlazhkoPeak(fZ, pZ)

        # period analysis
        Plinear = round(Plinear, 6)
        Pztf = round(Pztf, 6)
        Pmean = round((Plinear+Pztf)/2, 4)
        Pratio = round((Pztf/Plinear), 4)

        # saving the periodograms
        periodograms.append((Lid, (fL, pL, fFoldedL, pFoldedL), (fZ, pZ, fFoldedZ, pFoldedZ)))

        # fitting the light curves
        ntermsModels = 6

        if tZ.size==0 or mZ.size == 0 or meZ.size == 0 or Plinear == 0.0 or Pztf == 0.0 or tZ.size<40 or tL.size<40:
            LINEAR_Plinear = {
                'modelPhaseGrid': np.array(()), 
                'modelFit': np.array(()), 
                'dataPhasedTime': np.array(()), 
                'A': 0.0, 
                'mmax': 0.0, 
                'modTemplate': np.array(()), 
                'dataTemplate': np.array(()), 
                'dataTemplateErr': np.array(()), 
                'modelFit2data': np.array(()), 
                'rms': 0.0, 
                'chi': 0.0, 
                'chi2dof': 0.0, 
                'chi2dofR': 0.0
            }
            LINEAR_Pmean = {
                'modelPhaseGrid': np.array(()), 
                'modelFit': np.array(()), 
                'dataPhasedTime': np.array(()), 
                'A': 0.0, 
                'mmax': 0.0, 
                'modTemplate': np.array(()), 
                'dataTemplate': np.array(()), 
                'dataTemplateErr': np.array(()), 
                'modelFit2data': np.array(()), 
                'rms': 0.0, 
                'chi': 0.0, 
                'chi2dof': 0.0, 
                'chi2dofR': 0.0
            }
            ZTF_Pztf = {
                'modelPhaseGrid': np.array(()), 
                'modelFit': np.array(()), 
                'dataPhasedTime': np.array(()), 
                'A': 0.0, 
                'mmax': 0.0, 
                'modTemplate': np.array(()), 
                'dataTemplate': np.array(()), 
                'dataTemplateErr': np.array(()), 
                'modelFit2data': np.array(()), 
                'rms': 0.0, 
                'chi': 0.0, 
                'chi2dof': 0.0, 
                'chi2dofR': 0.0
            }
            ZTF_Pmean = {
                'modelPhaseGrid': np.array(()), 
                'modelFit': np.array(()), 
                'dataPhasedTime': np.array(()), 
                'A': 0.0, 
                'mmax': 0.0, 
                'modTemplate': np.array(()), 
                'dataTemplate': np.array(()), 
                'dataTemplateErr': np.array(()), 
                'modelFit2data': np.array(()), 
                'rms': 0.0, 
                'chi': 0.0, 
                'chi2dof': 0.0, 
                'chi2dofR': 0.0
            }
        else:
            LINEAR_Plinear = LCanalysisFromP(tL, mL, meL, Plinear, ntermsModels)
            LINEAR_Pmean = LCanalysisFromP(tL, mL, meL, Pmean, ntermsModels)


            ZTF_Pztf = LCanalysisFromP(tZ, mZ, meZ, Pztf, ntermsModels)
            ZTF_Pmean = LCanalysisFromP(tZ, mZ, meZ, Pmean, ntermsModels)

        STAR = [Plinear, Pztf, Pmean, Pratio, np.size(tL), LINEAR_Plinear['rms'], round(LINEAR_Plinear['chi2dof'], 1), round(LINEAR_Plinear['chi2dofR'], 1),LINEAR_Pmean['rms'],
            round(LINEAR_Pmean['chi2dof'],1), round(LINEAR_Pmean['chi2dofR'],1), round(LINEAR_Plinear['mmax'],2), round(LINEAR_Plinear['A'],2),
            np.size(tZ), ZTF_Pztf['rms'], round(ZTF_Pztf['chi2dof'],1), round(ZTF_Pztf['chi2dofR'],1), ZTF_Pmean['rms'], round(ZTF_Pmean['chi2dof'],1), round(ZTF_Pmean['chi2dofR'],1), round(ZTF_Pztf['mmax'],2), round(ZTF_Pztf['A'],2),
            round(fMainPeakL,6), round(fBlazhkoPeakL,6), round(BlazhkoPeriodL,6), round(BpowerRatioL,6), round(BsignificanceL,6), round(fMainPeakZ,6), 
            round(fBlazhkoPeakZ,6), round(BlazhkoPeriodZ,6), round(BpowerRatioZ,6), round(BsignificanceZ)]
            
        lc_analysis[Lid] = STAR
        fits.append((Lid, (LINEAR_Plinear, LINEAR_Pmean, ZTF_Pztf, ZTF_Pmean)))

    if (i % 10) == 0:
        # save dictionary to pkl file
        with open('../outputs/lc_analysis_'+end+'.pkl', 'wb') as fp:
            pickle.dump(lc_analysis, fp)
        with open("../outputs/periodograms_"+end+".pkl", "wb") as x:
            pickle.dump(periodograms, x)
        FITS = np.array(fits, dtype=object)
        np.save("../outputs/fits_"+end+".npy", FITS, allow_pickle=True)
        ZTF_data_lca = np.array(ZTF_data_lc, dtype=object)
        np.save("../outputs/ztf_data_"+end+".npy", ZTF_data_lca, allow_pickle=True)
        
with open('../outputs/lc_analysis_'+end+'.pkl', 'wb') as f:
    pickle.dump(lc_analysis, f)
with open("../outputs/periodograms_"+end+".pkl", "wb") as y:
        pickle.dump(periodograms, y)

FITS = np.array(fits, dtype=object)
np.save("../outputs/fits_"+end+".npy", FITS, allow_pickle=True)

ZTF_data_lca = np.array(ZTF_data_lc, dtype=object)
np.save("../outputs/ztf_data_"+end+".npy", ZTF_data_lca, allow_pickle=True)

  3%|▎         | 74/2710 [29:51<16:36:13, 22.68s/it]/Users/emadonev/Documents/correct var stars/var_stars/nbs/../src/blazhko_analysis.py:509: RuntimeWarning: divide by zero encountered in scalar divide
  powerFar = pFolded[fFolded>fBlazhkoPeak]  # frequencies beyond the second peak
  4%|▍         | 103/2710 [40:35<17:07:30, 23.65s/it]


AttributeError: 'numpy.ndarray' object has no attribute 'to_numpy'

***

In [32]:
cols = ['Plinear', 'Pztf', 'Pmean', 'Pratio', 'NdataLINEAR', 
        'L_rms', 'L_chi2dof', 'L_chi2dofR', 
        'Lmean_rms', 'Lmean_chi2dof', 'Lmean_chi2dofR', 'Lmmax', 'Lampl', 
        'NdataZTF', 'Z_rms', 'Zchi2dof', 
        'Zchi2dofR', 'Zmean_rms', 'Zmean_chi2dof', 'Zmean_chi2dofR', 
        'Zmmax', 'Zampl',
        'MainPeakL', 'BlazhkoPeakL', 'BlazhkoPeriodL','BpowerRatioL','BsignificanceL',
        'MainPeakZ', 'BlazhkoPeakZ', 'BlazhkoPeriodZ', 'BpowerRatioZ', 'BsignificanceZ']
LC_ANALYSIS = pd.DataFrame.from_dict(lc_analysis, orient='index', columns=cols)
LC_ANALYSIS.index.name='LINEAR id'
#LC_ANALYSIS = LC_ANALYSIS.drop(["what"], axis=1)
#LC_ANALYSIS = LC_ANALYSIS.drop(["what 2"], axis=1)
LC_ANALYSIS = LC_ANALYSIS.reset_index()
LC_ANALYSIS.to_csv("../outputs/LC_ANALYSIS_FINAL.csv", index=False)

In [35]:
LC_ANALYSIS.head()

,LINEAR id,Plinear,Pztf,Pmean,Pratio,NdataLINEAR,L_rms,L_chi2dof,L_chi2dofR,Lmean_rms,...,MainPeakL,BlazhkoPeakL,BlazhkoPeriodL,BpowerRatioL,BsignificanceL,MainPeakZ,BlazhkoPeakZ,BlazhkoPeriodZ,BpowerRatioZ,BsignificanceZ
0,29848,0.557020,0.557035,0.5570,1.0,301,0.000000,0.0,0.0,0.000000,...,1.795266,1.832831,26.620524,0.045499,7.483869,1.795220,1.858980,15.683814,0.206765,5
1,50402,0.643303,0.643294,0.6433,1.0,284,0.105976,0.6,0.7,0.107611,...,1.554479,1.622344,14.735136,0.077603,7.194603,1.554500,1.591735,26.856452,0.003194,10
2,62892,0.530776,0.530786,0.5308,1.0,276,0.108535,1.1,0.9,0.131104,...,1.884034,1.951914,14.731880,0.065840,7.123299,1.883999,1.943289,16.866251,0.004563,12
3,91437,0.674733,0.674737,0.6747,1.0,177,0.040739,2.8,1.3,0.073285,...,1.482068,1.549828,14.757969,0.081603,4.427005,1.482060,1.495845,72.542619,0.006555,11
4,95250,0.313870,0.313877,0.3139,1.0,222,0.162522,0.8,0.8,0.865595,...,3.186029,3.256529,14.184397,0.037702,9.998570,3.185965,3.188875,343.642612,0.003358,19


# Splitting the RR Lyrae into AB and C types

After we have computed everything necessary for RR Lyrae, we need to split them into AB and C types, since the Blazhko effect might be differently distributed amongst the different RR Lyrae types.

In [65]:
lc_data['LC TYPE'] = lc_type
lc_data.head()

,LINEAR id,Plinear,Pztf,Pmean,Pratio,NdataLINEAR,LPlin_rmsdmag,LPlin_chi2dof,LPlin_chi2dofR,LPmean_rmsdmag,...,ZPmean_rmsdmag,ZPmean_chi2dof,ZPmean_chi2dofR,Zmmax,Zampl,Lindicator,Llimit,Zindicator,Zlimit,LC TYPE
0,29848,0.557021,0.557037,0.557029,1.000028,301.0,0.178198,3.028026,1.451336,0.185326,...,0.052598,16.195588,3.955532,15.756993,0.925157,0,0.25,0.0,0.25,1
1,50402,0.643301,0.643296,0.643298,0.999992,284.0,0.107683,0.595738,0.733869,0.106092,...,0.031558,2.097207,1.177316,16.088901,0.486372,0,0.25,0.0,0.25,1
2,62892,0.530776,0.530785,0.530781,1.000017,276.0,0.108544,1.058829,0.934727,0.114002,...,0.069562,19.794606,3.288505,15.961735,0.641651,1,0.25,0.0,0.25,1
3,91437,0.674732,0.674738,0.674735,1.000009,177.0,0.040798,2.775415,1.352923,0.040888,...,0.020152,4.243845,1.580792,15.002323,0.864724,0,0.25,0.0,0.25,1
4,95250,0.313870,0.313876,0.313873,1.000019,222.0,0.162040,0.816590,0.859616,0.165801,...,0.054686,3.709448,1.515406,16.680144,0.461509,1,0.25,0.0,0.25,2


In [69]:
RRLab = lc_data[lc_data['LC TYPE']==1]
RRLc = lc_data[lc_data['LC TYPE']==2]
print(RRLab.shape)
print(RRLc.shape)

(2144, 28)
(566, 28)


In [70]:
RRLab.to_csv('../outputs/RRLAB.csv', index=False)
RRLc.to_csv('../outputs/RRLC.csv', index=False)

With the computations complete and all the data saved, it is time for the next step: analysis of RR Lyrae stars and the search for the Blazhko effect.